### **Spatial count, LDA, Clustering, RCN visualization, CSV Export and RCN correlations**  

**Author:** Alva Grönholm 
**Creation Date:** 30.11.2024 
**Modification Date:** 27.02.2025  

#### **🔹 Purpose**  
This notebook computes **spatial counts and LDA**, performs KMeans clustering based on the elbow curve on the spatial count and LDA results, visualizes results (RCNs) from clustering, and plots RCN correlations between the two methods. The clustered spatial count and LDA data is saved as a CSV file for downstream analysis.  

### **🔹 Workflow Steps**  

1. **Load Data:** Reads `.csv` file and creates an anndata object. Optionally already computed spatial count and LDA data can be loaded.

2. **Spatial count:** Computes the **spatial counts**, plots **elbow curves** and uses **Kneelocator** to find the optimal number of clusters. Performs **KMeans clustering** on the spatial count data and plots the results as **barplots**.   

3. **LDA:** Computes the **LDA**, plots **elbow curves** and uses **Kneelocator** to find the optimal number of clusters. Performs **KMeans clustering** on the LDA data and plots the results as **barplots**.

4. **Visualization:**  
   - **Spatial count** RCNs for different `radii`
   - **LDA** RCNs for different `radii`
   - **Phenotypes** 
   
---



In [1]:
#Loading libraries
import scimap as sm
import anndata as ad
import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree
from joblib import Parallel, delayed
import scipy
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer
import multiprocessing as mp
import seaborn as sns
from bokeh.plotting import figure, output_file, show 
from bokeh.palettes import Category10, Category20, Category20b, Category20
from scipy.stats import pearsonr
from statsmodels.stats.multitest import multipletests

Running SCIMAP  2.1.3


c:\Users\Localadmin_kangzigi\.conda\envs\scimap_nki\lib\site-packages\tqdm\auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



# Load data

In [4]:
#Split into data and meta
marker_list= ['CyclinE', 'pRb', 'CyclinA', 'CD11c', 'p27','Vimentin', 'DNA5', 'TIM-3', 'PanCK', 'Wee1', 'CD163', 'CyclinD1', 'p21', 'pH3', 'CD45',
              'MPM-2', 'pRPA', 'pStat1', 'CD20', 'PCNA', 'Geminin', 'gH2Ax', 'CD4', 'aSMA', 'CD8a', 'Iba1', 'PAX8', 'Ki67', 'FOX-P3', 'CD31', 'CD4']

data = df[marker_list]
meta = df.drop(columns=marker_list)

In [4]:
#Split into data and meta
marker_list= ['CD11c', 'Vimentin', 'PanCK', 'CD163', 'CD45', 'CD4', 'aSMA', 'CD8a', 'Iba1', 'PAX8', 'FOX-P3', 'CD31', 'CD20']

data = df[marker_list]
meta = df.drop(columns=marker_list)

In [5]:
adata = ad.AnnData (data)
adata.obs = meta

c:\Users\Localadmin_kangzigi\.conda\envs\scimap_nki\lib\site-packages\anndata\_core\aligned_df.py:67: ImplicitModificationWarning:

Transforming to str index.



In [6]:
# Create a globally unique cell ID by combining 'imageid' and 'CellID'
adata.obs['Global_CellID'] = adata.obs['imageid'].astype(str) + "_" + adata.obs['CellID'].astype(str)

# Set this new identifier as the index
adata.obs.set_index('Global_CellID', inplace=True)

# Exploration of markers for phenotyping

In [ ]:
# Heatmap 
sm.pl.heatmap(adata, groupBy='phenotype', standardScale='row', figsize=(10,5), showPrevalence=True, cmap = 'coolwarm', vmin=-2, vmax = 2)

# Spatial count 

In [ ]:
# Analyze spatial relationships using the radius method

# 40px = 13 micrometer
adata = sm.tl.spatial_count(adata, x_coordinate='X_centroid', y_coordinate='Y_centroid', imageid='imageid', phenotype='phenotype', method='radius', radius=40, label='spatial_count_40')
# 77px = 25 micrometer
adata = sm.tl.spatial_count(adata, x_coordinate='X_centroid', y_coordinate='Y_centroid', imageid='imageid', phenotype='phenotype', method='radius', radius=77, label='spatial_count_77')
# 108px = 35 micrometer
adata = sm.tl.spatial_count(adata, x_coordinate='X_centroid', y_coordinate='Y_centroid', imageid='imageid', phenotype='phenotype', method='radius', radius=108, label='spatial_count_108')

### Spatial count with results in counts (not percentage)

In [33]:
##Function to count the number of knn cells or cells in a radious
#It returns the actual number of cells of each cell_type, instead of the fraction of neighboors of each cell_type
def spatial_count_internal (adata_subset,x_coordinate,y_coordinate,phenotype,method,radius,knn,
                                subset,label,imageid):
        # Create a DataFrame with the necessary inforamtion
        data = pd.DataFrame({'x': adata_subset.obs[x_coordinate], 'y': adata_subset.obs[y_coordinate], 'phenotype': adata_subset.obs[phenotype]})

        # Identify neighbourhoods based on the method used
        # a) KNN method
        if method == 'knn':
            print("Identifying the " + str(knn) + " nearest neighbours for every cell")
            tree = BallTree(data[['x','y']], leaf_size= 2)
            ind = tree.query(data[['x','y']], k=knn, return_distance= False)
            neighbours = pd.DataFrame(ind.tolist(), index = data.index) # neighbour DF
            neighbours.drop(0, axis=1, inplace=True) # Remove self neighbour

        # b) Local radius method
        if method == 'radius':
            print("Identifying neighbours within " + str(radius) + " pixels of every cell")
            kdt = BallTree(data[['x','y']], metric='euclidean') 
            ind = kdt.query_radius(data[['x','y']], r=radius, return_distance=False)
            for i in range(0, len(ind)): ind[i] = np.delete(ind[i], np.argwhere(ind[i] == i))#remove self
            neighbours = pd.DataFrame(ind.tolist(), index = data.index) # neighbour DF

        # Map phenotype
        phenomap = dict(zip(list(range(len(ind))), data['phenotype'])) # Used for mapping

        # Loop through (all functionized methods were very slow)
        for i in neighbours.columns:
            neighbours[i] = neighbours[i].dropna().map(phenomap, na_action='ignore')

        # Drop NA
        #n_dropped = neighbours.dropna(how='all')

        # Collapse all the neighbours into a single column
        n = pd.DataFrame(neighbours.stack(), columns = ["neighbour_phenotype"])
        n.index = n.index.get_level_values(0) # Drop the multi index
        n = pd.DataFrame(n)
        n['order'] = list(range(len(n)))

        # Merge with real phenotype
        n_m = n.merge(data['phenotype'], how='inner', left_index=True, right_index=True)
        n_m['neighbourhood'] = n_m.index
        n = n_m.sort_values(by=['order'])

        # Normalize based on total cell count
        k = n.groupby(['neighbourhood','neighbour_phenotype']).size().unstack().fillna(0)
        #k = k.div(k.sum(axis=1), axis=0)

        # return the normalized neighbour occurance count
        return k
    
def spatial_count2 (adata,
                   x_coordinate='X_centroid',
                   y_coordinate='Y_centroid',
                   phenotype='phenotype',
                   method='radius',
                   radius=30,knn=10,
                   imageid='imageid',
                   subset=None,
                   label='spatial_count'):
    
    # Subset a particular image if needed
    if subset is not None:
        adata_list = [adata[adata.obs[imageid] == subset]]
    else:
        adata_list = [adata[adata.obs[imageid] == i] for i in adata.obs[imageid].unique()]

    # Apply function to all images and create a master dataframe
    # Create lamda function 
    r_spatial_count_internal = lambda x: spatial_count_internal(adata_subset=x,x_coordinate=x_coordinate,
                                                   y_coordinate=y_coordinate,phenotype=phenotype,
                                                   method=method,radius=radius,knn=knn,
                                                   imageid=imageid,subset=subset,label=label) 
    all_data = list(map(r_spatial_count_internal, adata_list)) # Apply function 


    # Merge all the results into a single dataframe    
    result = []
    for i in range(len(all_data)):
        result.append(all_data[i])
    result = pd.concat(result, join='outer')  

    # Reindex the cells
    result = result.fillna(0)
    result = result.reindex(adata.obs.index)

    # Add to adata
    adata.uns[label] = result

    # Return        
    return adata

In [ ]:
# Spatial count with radiuses 40, 77 and 108 px - returning actual number of neighbouring cells

#For radious of 40px (13 microns)
adata = spatial_count2(adata, x_coordinate='X_centroid', y_coordinate='Y_centroid', phenotype='phenotype', method='radius', radius=40, subset=None, label='spatial_count_40_count', knn=None)
#For radious of 77px (25 microns)
adata = spatial_count2(adata, x_coordinate='X_centroid', y_coordinate='Y_centroid', phenotype='phenotype', method='radius', radius=77, subset=None, label='spatial_count_77_count', knn=None)
#For radious of 108px (35 microns)
adata = spatial_count2(adata, x_coordinate='X_centroid', y_coordinate='Y_centroid', phenotype='phenotype', method='radius', radius=108, subset=None, label='spatial_count_108_count', knn=None)

In [ ]:
# Function to prepare data for combined distribution plot
def prepare_data(df, phenotype_df, phenotype_filter, label):
    filtered_df = df[phenotype_df["phenotype"].isin(phenotype_filter)]
    total_neighbors = filtered_df.sum(axis=1)  # Sum across columns to get total neighbors per cell
    return pd.DataFrame({"Total Neighbors": total_neighbors, "Radius": label, "Phenotype": phenotype_filter[0] if len(phenotype_filter) == 1 else " & ".join(phenotype_filter)})

# Extract and convert to DataFrame
df_phenotype = pd.DataFrame(adata.obs["phenotype"])  # Extract phenotype information
df_40 = pd.DataFrame(adata.uns["spatial_count_40_count"])
df_77 = pd.DataFrame(adata.uns["spatial_count_77_count"])
df_108 = pd.DataFrame(adata.uns["spatial_count_108_count"])

# Define phenotype groups
phenotype_groups = {
    "Tumor": ["Tumor"],
    "Stroma": ["Stromal"],
    "CD4+ & CD8+ T Cells": ["CD8_T_cells", "CD4_T_cells"],
    "CD163+ & CD163- Macrophages": ["CD163_pos_Macrophages", "CD163_neg_Macrophages"]
}

# Prepare data for each phenotype
combined_data = {}
for phenotype_label, phenotype_filter in phenotype_groups.items():
    data = []
    data.append(prepare_data(df_40, df_phenotype, phenotype_filter, "13µm"))
    data.append(prepare_data(df_77, df_phenotype, phenotype_filter, "25µm"))
    data.append(prepare_data(df_108, df_phenotype, phenotype_filter, "35µm"))
    combined_data[phenotype_label] = pd.concat(data)

# Set up separate boxplots for each phenotype
sns.set_style("whitegrid")
fig, axes = plt.subplots(2, 2, figsize=(8, 7))
axes = axes.flatten()

custom_colors = ["#AED581", "#FFB877", "#C39BD3"] 

for i, (phenotype_label, df) in enumerate(combined_data.items()):
    sns.boxplot(x="Radius", y="Total Neighbors", data=df, palette=custom_colors, ax=axes[i], showfliers=False)
    #medians = df.groupby("Radius")["Total Neighbors"].median()
    #for j, median in enumerate(medians): axes[i].text(j, median + 1, f'{median:.1f}', ha='center', color='black', fontsize=10)
    
    axes[i].set_title(f"{phenotype_label}")
    axes[i].set_xlabel("")
    axes[i].set_ylabel("Total neighbor count")
    axes[i].tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.savefig("", format="pdf", bbox_inches="tight")  # Save as PDF
plt.show()


In [ ]:
# Function to plot neighboring cell distributions
def plot_neighbor_counts(df, title):
    total_neighbors = df.sum(axis=1)  # Sum across columns to get total neighbors per cell
    unique, counts = np.unique(total_neighbors, return_counts=True)  # Count occurrences
    
    plt.figure(figsize=(25, 6))
    sns.barplot(x=unique, y=counts, color="skyblue")

    # Annotate bars with actual counts
    for i, count in enumerate(counts):
        plt.text(i, count + 2, str(count), ha='center', fontsize=8)

    plt.xlabel("Total Number of Neighboring Cells")
    plt.ylabel("Count")
    plt.title(title)
    plt.xticks(rotation=45)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

# Extract and convert to DataFrame
df_40 = pd.DataFrame(adata.uns["spatial_count_40_count"]) 
df_77 = pd.DataFrame(adata.uns["spatial_count_77_count"])
df_108 = pd.DataFrame(adata.uns["spatial_count_108_count"])

# Plot distributions
plot_neighbor_counts(df_40, "Neighbor Count Distribution (40px, 13 microns)")
plot_neighbor_counts(df_77, "Neighbor Count Distribution (77px, 25 microns)")
plot_neighbor_counts(df_108, "Neighbor Count Distribution (108px, 35 microns)")


## Clustering of spatial count results

### Elbow curves

In [ ]:
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer
from kneed import KneeLocator
import numpy as np
import matplotlib.pyplot as plt

# Define the range of k values
k_range = range(2, 56)

# Fit KMeans and collect distortion values
distortions = []
for k in k_range:
    km = KMeans(n_clusters=k, random_state=20)
    km.fit(adata.uns['spatial_count_40'].fillna(0))
    distortions.append(km.inertia_)  # Inertia = distortion score

# Use KneeLocator to find the optimal k
kl = KneeLocator(k_range, distortions, curve="convex", direction="decreasing")

# Print the optimal k
print(f"Optimal k found: {kl.elbow}")

# Plot elbow curve with knee marked
plt.figure(figsize=(8, 6))
plt.plot(k_range, distortions, 'bo-', markersize=5, label="Distortion")
plt.axvline(x=kl.elbow, color='r', linestyle="--", label=f"Optimal k = {kl.elbow}")
plt.xlabel("Number of Clusters (k)")
plt.ylabel("Distortion (Inertia)")
plt.title("Elbow Curve with KneeLocator, spatial count 13 microns")
plt.legend()
# Save as PDF
plt.savefig("", format="pdf")

plt.show()


In [ ]:
# Define the range of k values
k_range = range(2, 56)

# Fit KMeans and collect distortion values
distortions = []
for k in k_range:
    km = KMeans(n_clusters=k, random_state=20)
    km.fit(adata.uns['spatial_count_77'].fillna(0))
    distortions.append(km.inertia_)  # Inertia = distortion score

# Use KneeLocator to find the optimal k
kl = KneeLocator(k_range, distortions, curve="convex", direction="decreasing")

# Print the optimal k
print(f"Optimal k found: {kl.elbow}")

# Plot elbow curve with knee marked
plt.figure(figsize=(8, 6))
plt.plot(k_range, distortions, 'bo-', markersize=5, label="Distortion")
plt.axvline(x=kl.elbow, color='r', linestyle="--", label=f"Optimal k = {kl.elbow}")
plt.xlabel("Number of Clusters (k)")
plt.ylabel("Distortion (Inertia)")
plt.title("Elbow Curve with KneeLocator, spatial count 25 microns")
plt.legend()
# Save as PDF
plt.savefig("", format="pdf")
plt.show()


In [ ]:
# Define the range of k values
k_range = range(2, 56)

# Fit KMeans and collect distortion values
distortions = []
for k in k_range:
    km = KMeans(n_clusters=k, random_state=20)
    km.fit(adata.uns['spatial_count_108'].fillna(0))
    distortions.append(km.inertia_)  # Inertia = distortion score

# Use KneeLocator to find the optimal k
kl = KneeLocator(k_range, distortions, curve="convex", direction="decreasing")

# Print the optimal k
print(f"Optimal k found: {kl.elbow}")

# Plot elbow curve with knee marked
plt.figure(figsize=(8, 6))
plt.plot(k_range, distortions, 'bo-', markersize=5, label="Distortion")
plt.axvline(x=kl.elbow, color='r', linestyle="--", label=f"Optimal k = {kl.elbow}")
plt.xlabel("Number of Clusters (k)")
plt.ylabel("Distortion (Inertia)")
plt.title("Elbow Curve with KneeLocator, spatial count 35 microns")
plt.legend()
# Save as PDF
plt.savefig("", format="pdf")
plt.show()


### KMeans clustering

In [ ]:
# Apply K-Means clustering, k based on elbow curve the optimal k from kneelocator

# Radius 40px
adata = sm.tl.spatial_cluster(adata, df_name="spatial_count_40", method='kmeans', k=15, random_state=22, label='spatial_count_40_k15')
# Radius 77px
adata = sm.tl.spatial_cluster(adata, df_name="spatial_count_77", method='kmeans', k=12, random_state=22, label='spatial_count_77_k12')
# Radius 108px
adata = sm.tl.spatial_cluster(adata, df_name="spatial_count_108", method='kmeans', k=13, random_state=22, label='spatial_count_108_k13')

In [ ]:
# Apply K-Means clustering, k=10 based on elbow curve

# Radius 40px
adata = sm.tl.spatial_cluster(adata, df_name="spatial_count_40", method='kmeans', k=10, random_state=22, label='spatial_count_40_k10')
# Radius 77px
adata = sm.tl.spatial_cluster(adata, df_name="spatial_count_77", method='kmeans', k=10, random_state=22, label='spatial_count_77_k10')
# Radius 108px
adata = sm.tl.spatial_cluster(adata, df_name="spatial_count_108", method='kmeans', k=10, random_state=22, label='spatial_count_108_k10')

### Barplots of K-Means clustering results

In [ ]:
# Barplot of phenotype proportions in clusters
sm.pl.stacked_barplot(adata, x_axis='spatial_count_40_k15', y_axis='phenotype', subset_xaxis=None, subset_yaxis=None, order_xaxis=None, order_yaxis=None, method='percent', plot_tool='matplotlib', matplotlib_cmap=None, matplotlib_bbox_to_anchor=(1, 1.02), matplotlib_legend_loc=2, return_data=False)

In [ ]:
# Barplot of phenotype proportions in clusters
sm.pl.stacked_barplot(adata, x_axis='spatial_count_77_k12', y_axis='phenotype', subset_xaxis=None, subset_yaxis=None, order_xaxis=None, order_yaxis=None, method='percent', plot_tool='matplotlib', matplotlib_cmap=None, matplotlib_bbox_to_anchor=(1, 1.02), matplotlib_legend_loc=2, return_data=False)

In [ ]:
# Barplot of phenotype proportions in clusters
sm.pl.stacked_barplot(adata, x_axis='spatial_count_108_k13', y_axis='phenotype', subset_xaxis=None, subset_yaxis=None, order_xaxis=None, order_yaxis=None, method='percent', plot_tool='matplotlib', matplotlib_cmap=None, matplotlib_bbox_to_anchor=(1, 1.02), matplotlib_legend_loc=2, return_data=False)

In [ ]:
# Barplot of proportion of cluster in each sample
sm.pl.stacked_barplot(adata, x_axis='imageid', y_axis='spatial_count_77_k10', subset_xaxis=None, subset_yaxis=None, order_xaxis=None, order_yaxis=None, method='percent', plot_tool='matplotlib', matplotlib_cmap=None, matplotlib_bbox_to_anchor=(1, 1.02), matplotlib_legend_loc=2, return_data=False)

# Spatial LDA

In [ ]:
# LDA, number of latent_motifs = 10

# Radius 40px (13 microns)
adata = sm.tl.spatial_lda(adata, method='radius', radius=40, label='spatial_lda_40', random_state=8)
# Radius 77px (25 microns)
adata = sm.tl.spatial_lda(adata, method='radius', radius=77, label='spatial_lda_77', random_state=10)
# Radius 108px (35 microns)
adata = sm.tl.spatial_lda(adata, method='radius', radius=108, label='spatial_lda_108', random_state=12)

## Clustering of LDA results

### Elbow curves

In [ ]:
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer
from kneed import KneeLocator
import numpy as np
import matplotlib.pyplot as plt

# Define the range of k values
k_range = range(2, 56)

# Fit KMeans and collect distortion values
distortions = []
for k in k_range:
    km = KMeans(n_clusters=k, random_state=20)
    km.fit(adata.uns['spatial_lda_40'].fillna(0))
    distortions.append(km.inertia_)  # Inertia = distortion score

# Use KneeLocator to find the optimal k
kl = KneeLocator(k_range, distortions, curve="convex", direction="decreasing")

# Print the optimal k
print(f"Optimal k found: {kl.elbow}")

# Plot elbow curve with knee marked
plt.figure(figsize=(8, 6))
plt.plot(k_range, distortions, 'bo-', markersize=5, label="Distortion")
plt.axvline(x=kl.elbow, color='r', linestyle="--", label=f"Optimal k = {kl.elbow}")
plt.xlabel("Number of Clusters (k)")
plt.ylabel("Distortion (Inertia)")
plt.title("Elbow Curve with KneeLocator")
plt.legend()
# Save as PDF
plt.savefig("", format="pdf")
plt.show()

In [ ]:
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer
from kneed import KneeLocator
import numpy as np
import matplotlib.pyplot as plt# Define the range of k values
k_range = range(2, 56)

# Fit KMeans and collect distortion values
distortions = []
for k in k_range:
    km = KMeans(n_clusters=k, random_state=20)
    km.fit(adata.uns['spatial_lda_77'].fillna(0))
    distortions.append(km.inertia_)  # Inertia = distortion score

# Use KneeLocator to find the optimal k
kl = KneeLocator(k_range, distortions, curve="convex", direction="decreasing")

# Print the optimal k
print(f"Optimal k found: {kl.elbow}")

# Plot elbow curve with knee marked
plt.figure(figsize=(8, 6))
plt.plot(k_range, distortions, 'bo-', markersize=5, label="Distortion")
plt.axvline(x=kl.elbow, color='r', linestyle="--", label=f"Optimal k = {kl.elbow}")
plt.xlabel("Number of Clusters (k)")
plt.ylabel("Distortion (Inertia)")
plt.title("Elbow Curve with KneeLocator (LDA 25 microns)")
plt.legend()
# Save as PDF
plt.savefig("F:/Alva/Final_spatial_analysis_plots/elbow_lda_25_microns_2.pdf", format="pdf")
plt.show()

In [ ]:
# Define the range of k values
k_range = range(2, 56)

# Fit KMeans and collect distortion values
distortions = []
for k in k_range:
    km = KMeans(n_clusters=k, random_state=20)
    km.fit(adata.uns['spatial_lda_108'].fillna(0))
    distortions.append(km.inertia_)  # Inertia = distortion score

# Use KneeLocator to find the optimal k
kl = KneeLocator(k_range, distortions, curve="convex", direction="decreasing")

# Print the optimal k
print(f"Optimal k found: {kl.elbow}")

# Plot elbow curve with knee marked
plt.figure(figsize=(8, 6))
plt.plot(k_range, distortions, 'bo-', markersize=5, label="Distortion")
plt.axvline(x=kl.elbow, color='r', linestyle="--", label=f"Optimal k = {kl.elbow}")
plt.xlabel("Number of Clusters (k)")
plt.ylabel("Distortion (Inertia)")
plt.title("Elbow Curve with KneeLocator")
plt.legend()
# Save as PDF
plt.savefig("F:/Alva/Final_spatial_analysis_plots/elbow_lda_35_microns_2.pdf", format="pdf")
plt.show()

### KMeans clustering

In [ ]:
# K-Means clustering of LDA, k=10 

# Radius 40 px 
adata = sm.tl.spatial_cluster(adata, df_name="spatial_lda_40", method='kmeans', k=13, random_state=10, label='lda_40_k13')
# Radius 40 px 
adata = sm.tl.spatial_cluster(adata, df_name="spatial_lda_77", method='kmeans', k=15, random_state=10, label='lda_77_k15')
# Radius 40 px 
adata = sm.tl.spatial_cluster(adata, df_name="spatial_lda_108", method='kmeans', k=13, random_state=10, label='lda_108_k13')

In [ ]:
# K-Means clustering of LDA, k=10 

# Radius 40 px 
adata = sm.tl.spatial_cluster(adata, df_name="spatial_lda_40", method='kmeans', k=10, random_state=10, label='lda_40_k10')
# Radius 40 px 
adata = sm.tl.spatial_cluster(adata, df_name="spatial_lda_77", method='kmeans', k=10, random_state=10, label='lda_77_k10')
# Radius 40 px 
adata = sm.tl.spatial_cluster(adata, df_name="spatial_lda_108", method='kmeans', k=10, random_state=10, label='lda_108_k10')

### Barplots of clustering results

In [ ]:
# Barplot of proportion of phenotypes in each LDA cluster
sm.pl.stacked_barplot(adata, x_axis='lda_40_k13', y_axis='phenotype', subset_xaxis=None, subset_yaxis=None, order_xaxis=None, order_yaxis=None, method='percent', plot_tool='matplotlib', matplotlib_cmap=None, matplotlib_bbox_to_anchor=(1, 1.02), matplotlib_legend_loc=2, return_data=False)

In [ ]:
# Barplot of proportion of phenotypes in each LDA cluster
sm.pl.stacked_barplot(adata, x_axis='lda_77_k15', y_axis='phenotype', subset_xaxis=None, subset_yaxis=None, order_xaxis=None, order_yaxis=None, method='percent', plot_tool='matplotlib', matplotlib_cmap=None, matplotlib_bbox_to_anchor=(1, 1.02), matplotlib_legend_loc=2, return_data=False)

In [ ]:
# Barplot of proportion of phenotypes in each LDA cluster
sm.pl.stacked_barplot(adata, x_axis='lda_108_k13', y_axis='phenotype', subset_xaxis=None, subset_yaxis=None, order_xaxis=None, order_yaxis=None, method='percent', plot_tool='matplotlib', matplotlib_cmap=None, matplotlib_bbox_to_anchor=(1, 1.02), matplotlib_legend_loc=2, return_data=False)

# Visualization

### Visualize RCNs on top of image

In [7]:
adata.obs['spatial_count_40_k15'] = adata.obs['spatial_count_40_k15'].astype(str)
adata.obs['spatial_count_77_k12'] = adata.obs['spatial_count_77_k12'].astype(str)
adata.obs['spatial_count_108_k13'] = adata.obs['spatial_count_108_k13'].astype(str)
adata.obs['lda_40_k13'] = adata.obs['lda_40_k13'].astype(str)
adata.obs['lda_77_k15'] = adata.obs['lda_77_k15'].astype(str)
adata.obs['lda_108_k13'] = adata.obs['lda_108_k13'].astype(str)

In [10]:
marker_list_with_DNA = ['DNA1','CyclinE','DNA2', 'pChk1', 'DNA3', 'Tcf1', 'pRb', 'CyclinB1', 'DNA4', 'CyclinA', 'CD11c', 'p27',
                          'Vimentin', 'DNA5', 'TIM-3', 'PanCK', 'Wee1', 'DNA6', 'CD163', 'CyclinD1', 'p21', 'DNA7', 'pH3', 'CD45',
                            'MPM-2', 'DNA8', 'pRPA', 'pStat1', 'CD20', 'DNA9', 'PCNA', 'Geminin', 'gH2Ax', 'DNA10', 'CD4', 'aSMA',
                            'CD8a', 'DNA11', 'Iba1', 'PAX8', 'PD-1', 'DNA12', 'Ki67', 'FOX-P3', 'CD31', 'DNA13', 'CD4', 'FAP','NKG2a']

In [48]:
colors = {"0" : "#db82b3", "1" : "#fdf2c0", "2" : "#a6c7d1", "3" : "#fedc80", "4" : "#fffd83", "5" : "#dba4c3", "6" : "#febd87", "7" : "#bc9fda", "8" : "#eadefe", "9" : "#BADE8C", '10' : '#6a3d9a', '11':'#ADD8E6', '12':'#e377c2', '13':'#b2df8a', '14': '#fb8072'}

In [29]:
colors = {'3': "#eadefe", '10' : "#fdf2c0", "9" : "#BADE8C"}

In [ ]:
sm.pl.image_viewer(image_path, adata, imageid = 'imageid', subset='', channel_names=marker_list_with_DNA, point_size=30, point_color = colors, overlay='lda_40_k13', x_coordinate='X_centroid', y_coordinate='Y_centroid')

In [ ]:
sm.pl.image_viewer(image_path, adata, imageid = 'imageid', subset='', channel_names=marker_list_with_DNA, point_size=30, point_color = colors, overlay='lda_40_k13', x_coordinate='X_centroid', y_coordinate='Y_centroid')

### Visualize phenotypes on top of image

In [ ]:
phenotype_colors = {"Tumor" : "#B1CCE3",       
  "CD4_T_cells" : "#DA3683",   
  "CD8_T_cells" : "#EAA09A",
  "B_cells" : "#045275",              
  "CD163_pos_DC" : "#F7D84D",             
  "DC" : "#BADE8C",                    
  "CD163_pos_Macrophages" : "#C6B4D5",    
  "CD163_neg_Macrophages" : "#C060BD",    
  "Stromal" : "#F7F9A3",              
  "Endothelial" : "#EB881F"}

In [32]:
phenotype_colors = {
     "Tumor": "#5A79A5",  # Darker blue
    "CD4_T_cells": "#DA3683",  # Dark magenta
    "FOXP3_CD4_T_cells": "#FFC2D7",  # Deeper pink
    "CD8_T_cells": "#9A1F5F",  # Darker red
    "B_cells": "#023E73",  # Deep navy blue
    "CD163_DC": "#C09A3D",  # Dark gold
    "DC": "#5C883A",  # Deep olive green
    "CD163_Macrophages": "#C060BD",  # Muted purple
    "CD163_Macrophages": "#7A1F87",  # Deep violet
    "Stromal": "#efeb00",  # Darker yellow-green
    "Endothelial": "#EB881F",  # Rich brown-orange
}


In [ ]:
sm.pl.image_viewer(image_path, adata, overlay = 'phenotype', point_color=phenotype_colors, subset='', channel_names = marker_list_with_DNA, x_coordinate="X_centroid",y_coordinate="Y_centroid", point_size=30)

In [ ]:
sm.pl.image_viewer(image_path, adata, overlay = 'phenotype', point_color=phenotype_colors, subset='', channel_names = marker_list_with_DNA, x_coordinate="X_centroid",y_coordinate="Y_centroid", point_size=30)

# Clustree

In [ ]:
import scanpy as sc
from pyclustree import clustree

custom_columns = ["spatial_count_40_k10", "spatial_count_77_k10", "spatial_count_108_k10"]  # Your column names

fig = clustree(
    adata,
    custom_columns,  # Pass the list directly
    title="Clustree of spatial count",
    edge_weight_threshold=0.05,
    show_fraction=True,
)
fig.set_size_inches(12, 7)
fig.set_dpi(300)